In [ ]:
import pandas as pd
from shapely.geometry import Polygon, Point, LineString
import folium
import random
import googlemaps
import json

# Load Google Maps API key from a protected file
# GOOGLE_MAPS_API_KEY: API key for Google Maps / Google Maps API kulcs
with open('config.json', 'r') as config_file:
    config = json.load(config_file)
GOOGLE_MAPS_API_KEY = config['GOOGLE_MAPS_API_KEY']

# DEBRECEN_POLYGON_MAP_COORDINATES_PATH: CSV file path containing Debrecen polygon coordinates
# DEBRECEN_POLYGON_MAP_COORDINATES_PATH: Debrecen poligon koordinátákat tartalmazó CSV fájl elérési útvonala
DEBRECEN_POLYGON_MAP_COORDINATES_PATH = 'coordinates/debrecen_coordinates_v1.csv'

# NO_FLY_ZONE_COORDINATES_PATHS: List of CSV file paths for no-fly zone coordinates (airport, clinics, etc.)
# NO_FLY_ZONE_COORDINATES_PATHS: Lista a tilos zónák koordinátáit tartalmazó CSV fájlok útvonalairól (repülőtér, klinikák, stb.)
NO_FLY_ZONE_COORDINATES_PATHS = [
    'coordinates/airport_coordinates_v1.csv',
    'coordinates/clinics_coordinates_v1.csv',
    'coordinates/kenezi_coordinates_v1.csv',
    'coordinates/mainsquare_coordinates_v1.csv',
    'coordinates/military_coordinates_v1.csv',
    'coordinates/railwaystation_coordinates_v1.csv',
    'coordinates/stadium_coordinates_v1.csv'
]

# NUMBER_OF_SIMULATED_ORDERS: Number of random orders to simulate / Szimulált rendelések száma
NUMBER_OF_SIMULATED_ORDERS = 500

# FORUM_COORDINATE_X and FORUM_COORDINATE_Y: Coordinates for Fórum Debrecen / Fórum Debrecen koordinátái
FORUM_COORDINATE_X = 47.5326511
FORUM_COORDINATE_Y = 21.6287677

# PLOT_ROUTES_ON_MAP: Flag to enable plotting of routes on the map / Jelző az útvonalak térképen való megjelenítéséhez
PLOT_ROUTES_ON_MAP = True

# Load Debrecen polygon coordinates
data = pd.read_csv(DEBRECEN_POLYGON_MAP_COORDINATES_PATH)
coordinates = list(zip(data['Latitude'], data['Longitude']))
debrecen_polygon = Polygon(coordinates)

# Load no-fly zones as polygons
no_fly_zones = []
for path in NO_FLY_ZONE_COORDINATES_PATHS:
    no_fly_data = pd.read_csv(path)
    no_fly_coordinates = list(zip(no_fly_data['Latitude'], no_fly_data['Longitude']))
    no_fly_zones.append(Polygon(no_fly_coordinates))

# Generate the base map centered on Debrecen
center_lat = data['Latitude'].mean()
center_lon = data['Longitude'].mean()
map_debrecen = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# Add Debrecen polygon to the map
folium.Polygon(
    locations=coordinates,
    color='blue',
    fill=True,
    fill_color='lightblue',
    fill_opacity=0.5,
    weight=2
).add_to(map_debrecen)

# Add no-fly zones to the map
for no_fly_zone in no_fly_zones:
    folium.Polygon(
        locations=list(no_fly_zone.exterior.coords),
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.5,
        weight=2
    ).add_to(map_debrecen)

# Generate random points inside the Debrecen polygon excluding no-fly zones
min_x, min_y, max_x, max_y = debrecen_polygon.bounds
random_points_inside_polygon = []

while len(random_points_inside_polygon) < NUMBER_OF_SIMULATED_ORDERS:
    random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
    if debrecen_polygon.contains(random_point) and not any(zone.contains(random_point) for zone in no_fly_zones):
        random_points_inside_polygon.append((random_point.x, random_point.y))

# Add random order points to the map
for point in random_points_inside_polygon:
    folium.CircleMarker(
        location=point,
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
    ).add_to(map_debrecen)

# Mark Fórum Debrecen on the map
forum_point = Point(FORUM_COORDINATE_X, FORUM_COORDINATE_Y)
forum_point_data = (forum_point.x, forum_point.y)
folium.CircleMarker(
    location=forum_point_data,
    radius=8,
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.9,
).add_to(map_debrecen)

# Initialize Google Maps API client
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

# Calculate routes and durations for each destination using various transportation modes
results = []
for idx, destination in enumerate(random_points_inside_polygon):
    modes = ['driving', 'walking', 'bicycling']
    route_info = {'Destination': idx + 1, 'Coordinates': destination}
    
    for mode in modes:
        directions = gmaps.directions(
            origin=forum_point_data,
            destination=destination,
            mode=mode
        )
        
        if directions:
            leg = directions[0]["legs"][0]
            route_info[f'{mode}_duration'] = leg["duration"]["text"]
            route_info[f'{mode}_distance'] = leg["distance"]["text"]
        else:
            route_info[f'{mode}_duration'] = 'N/A'
            route_info[f'{mode}_distance'] = 'N/A'
    
    results.append(route_info)

# Plot driving routes on the map if enabled
if PLOT_ROUTES_ON_MAP:
    map_plot = folium.Map(location=[FORUM_COORDINATE_X, FORUM_COORDINATE_Y], zoom_start=13)
    
    # Add Fórum Debrecen marker as the start point
    folium.Marker(location=forum_point_data, popup="Start Point", icon=folium.Icon(color="blue")).add_to(map_plot)
    
    # Add destination markers and driving routes for each order
    for result in results:
        folium.Marker(
            location=result['Coordinates'],
            popup=f"Destination {result['Destination']}",
            icon=folium.Icon(color="red")
        ).add_to(map_plot)
        
        directions = gmaps.directions(
            origin=forum_point_data,
            destination=result['Coordinates'],
            mode='driving'
        )
        
        if directions:
            steps = directions[0]['legs'][0]['steps']
            route_coordinates = [(step['start_location']['lat'], step['start_location']['lng']) for step in steps]
            route_coordinates.append(result['Coordinates'])
            folium.PolyLine(route_coordinates, color='green', weight=2.5, opacity=0.8).add_to(map_plot)
    
    map_plot.save("driving_routes_map.html")

# Save results to a DataFrame for further analysis
results_df = pd.DataFrame(results)

# Optionally, save results and maps to files
# results_df.to_csv("delivery_data.csv", sep=";")
# map_debrecen.save("delivery_data_map.html")

KeyboardInterrupt: 

In [ ]:
def calculate_length(coords):
    """
    English:
    Calculate the length of a LineString constructed from a list of (latitude, longitude) tuples.
    
    Hungarian:
    Kiszámolja a LineString hosszát, amelyet egy (szélességi, hosszúsági) pontok listájából hoz létre.
    """
    return LineString(coords).length

def get_detour_route(zone, entry, exit):
    """
    English:
    Computes a detour route along the boundary of a given polygon (zone) between the specified entry and exit points.
    It inserts the entry and exit points into the polygon's boundary if needed and returns the shorter of the two possible paths.
    
    Hungarian:
    Kiszámítja a kitérő útvonalat a megadott zóna (poligon) határán az adott belépési (entry) és kilépési (exit) pontok között.
    Szükség esetén beilleszti ezeket a pontokat a poligon határkoordinátái közé, majd a két lehetséges útvonal közül a rövidebbet adja vissza.
    """
    coords = list(zone.exterior.coords)
    
    def insert_point(coords, pt):
        """
        English:
        Inserts point 'pt' into the coordinate list if it lies on the segment between two consecutive points.
        
        Hungarian:
        Beilleszti a 'pt' pontot a koordináta listába, ha az két egymást követő pont által meghatározott szakaszra esik.
        """
        new_coords = []
        for i in range(len(coords) - 1):
            seg = LineString([coords[i], coords[i+1]])
            if seg.distance(pt) < 1e-8:
                new_coords.append(coords[i])
                if (pt.x, pt.y) != coords[i]:
                    new_coords.append((pt.x, pt.y))
            else:
                new_coords.append(coords[i])
        new_coords.append(coords[-1])
        if (pt.x, pt.y) not in new_coords:
            new_coords.append((pt.x, pt.y))
        return new_coords

    # Insert entry and exit points into the polygon boundary if necessary.
    new_coords = insert_point(coords, entry)
    new_coords = insert_point(new_coords, exit)
    
    try:
        idx_entry = new_coords.index((entry.x, entry.y))
    except ValueError:
        idx_entry = 0
    try:
        idx_exit = new_coords.index((exit.x, exit.y))
    except ValueError:
        idx_exit = 0
    
    if idx_entry <= idx_exit:
        candidate1 = new_coords[idx_entry:idx_exit+1]
    else:
        candidate1 = new_coords[idx_entry:] + new_coords[:idx_exit+1]
    
    if idx_exit <= idx_entry:
        candidate2 = new_coords[idx_exit:idx_entry+1]
    else:
        candidate2 = new_coords[idx_exit:] + new_coords[:idx_entry+1]
    
    candidate1_line = LineString(candidate1)
    candidate2_line = LineString(candidate2)
    
    return candidate1 if candidate1_line.length <= candidate2_line.length else candidate2

def compute_safe_route(start, end, zones):
    """
    English:
    Computes a safe route from a starting point to an ending point that avoids crossing any no-fly zones.
    It identifies intersections between the direct path and zone boundaries, calculates detour paths along those boundaries,
    and recursively constructs a safe route.
    
    Hungarian:
    Kiszámít egy biztonságos útvonalat a kezdő- és célpont között, amely elkerüli a tilozott zónák átszelését.
    Meghatározza a közvetlen útvonal és a zóna határainak metszéspontjait, kiszámítja a kitérő útvonalakat a határ mentén,
    és rekurzívan építi fel a biztonságos útvonalat.
    
    Returns:
        English:
            A list of (latitude, longitude) coordinate tuples forming the safe route.
        Hungarian:
            Egy (szélességi, hosszúsági) koordinátapárokat tartalmazó lista, amely a biztonságos útvonalat alkotja.
    """
    start_pt = Point(start)
    end_pt = Point(end)
    direct_line = LineString([start, end])
    
    intersections = []
    for zone in zones:
        if direct_line.intersects(zone):
            inter_geom = direct_line.intersection(zone.boundary)
            if inter_geom.is_empty:
                continue
            if inter_geom.geom_type == 'Point':
                pts = [inter_geom]
            elif inter_geom.geom_type == 'MultiPoint':
                pts = list(inter_geom.geoms)
            else:
                pts = []
            if len(pts) < 2:
                continue
            pts = sorted(pts, key=lambda p: start_pt.distance(p))
            entry_pt = pts[0]
            exit_pt = pts[-1]
            intersections.append({
                'zone': zone,
                'entry': entry_pt,
                'exit': exit_pt,
                'distance': start_pt.distance(entry_pt)
            })
    
    if not intersections:
        return [start, end]
    
    intersections.sort(key=lambda x: x['distance'])
    route = []
    current_start = start
    
    for inter in intersections:
        zone = inter['zone']
        entry_pt = inter['entry']
        exit_pt = inter['exit']
        safe_before = compute_safe_route(current_start, (entry_pt.x, entry_pt.y), zones)
        route.extend(safe_before[:-1])
        
        detour_coords = get_detour_route(zone, entry_pt, exit_pt)
        normal_detour = detour_coords
        reversed_detour = list(reversed(detour_coords))
        
        cost_normal = Point((entry_pt.x, entry_pt.y)).distance(Point(normal_detour[0])) + \
                        Point(normal_detour[-1]).distance(Point((exit_pt.x, exit_pt.y)))
        cost_reversed = Point((entry_pt.x, entry_pt.y)).distance(Point(reversed_detour[0])) + \
                        Point(reversed_detour[-1]).distance(Point((exit_pt.x, exit_pt.y)))
        
        chosen_detour = reversed_detour if cost_reversed < cost_normal else normal_detour
        route.extend(chosen_detour)
        current_start = (exit_pt.x, exit_pt.y)
    
    safe_after = compute_safe_route(current_start, end, zones)
    route.extend(safe_after[1:])
    return route

# Compute safe routes for each destination.
drone_routes = []
for dest in random_points_inside_polygon:
    route = compute_safe_route(forum_point_data, dest, no_fly_zones)
    drone_routes.append(route)

# Plot the computed safe drone routes.
for route in drone_routes:
    for i in range(len(route) - 1):
        segment = [route[i], route[i+1]]
        folium.PolyLine(segment, color='purple', weight=1, opacity=0.8).add_to(map_debrecen)

# Save the updated map.
map_debrecen.save("drone_routes_map.html")

# Optionally, display the routes in a DataFrame for inspection.
import pandas as pd
drone_routes_df = pd.DataFrame({
    'Destination': list(range(1, len(drone_routes) + 1)),
    'Route': drone_routes
})

Todo list
- költségkalkuláció kiszállítási módokhoz
- drónos kiszállítás költségkalkuláció
- drónos kiszállítás időkalkuláció
- nagy adathalmaz legenerálása
- modell alkotás
- előrejelző szkript megírása